***INSTALL REQUIREMENTS***

In [ ]:
# %pip install ultralytics==8.2.103 -q
# %pip install roboflow --quiet
# %pip install supervision==0.24.0 -q

***IMPORT STATION***

In [15]:
import ultralytics
from ultralytics import YOLO
from IPython.display import display, Image
import yaml
import supervision as sv
import cv2
from IPython.display import clear_output
import sqlite3
from datetime import datetime
import os

***TRAINING***

In [ ]:
!yolo settings sync=False

ultralytics.checks()

dataset = "/home/abe-tanu/Documents/Code/Python/Artificial-Intelligence/OBB AI PPTI 21 Team 2.v2i.yolov8-obb/datasets"

# Download the dataset
with open(f'{dataset}/data.yaml', 'r') as f:
    data = yaml.safe_load(f)
data['train'] = '../train/images'
data['val'] = '../valid/images'
data['test'] = '../test/images'
if 'path' in data:
  del data['path']
with open(f'{dataset}/data.yaml', 'w') as f:
    yaml.dump(data, f, sort_keys=False)

# Train the model
model = YOLO('yolov8n-obb.pt')
results = model.train(data=f"{dataset}/data.yaml", epochs=100, imgsz=640)


***DATABASE***

In [18]:
conn = sqlite3.connect("presensi.db")
c = conn.cursor()
c.execute('''
    CREATE TABLE IF NOT EXISTS presensi (
        nama TEXT,
        status TEXT,
        waktu TEXT
    )
''')
conn.commit()

***MODELING***

In [3]:
def sudah_terdeteksi(nama):
    c.execute("SELECT 1 FROM presensi WHERE nama=? AND status='Hadir'", (nama,))
    return c.fetchone() is not None

def tandai_hadir(nama):
    waktu = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    c.execute("INSERT INTO presensi (nama, status, waktu) VALUES (?, 'Hadir', ?)", (nama, waktu))
    conn.commit()

# Load model YOLOv8-OBB
model = YOLO("/home/abe-tanu/Documents/Code/Python/Artificial-Intelligence/presensi_obb/runs/obb/train/weights/best.pt")

# Daftar anak-anak
daftar_anak = ["Adriel Bernhard T", "Jonea Kristiawan", "Kevin Tanwiputra", "Kevin Jiovanni Kuslin"]

# Buka webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Tidak dapat membuka webcam")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Gagal membaca frame")
        break

    # Jalankan prediksi
    results = model.predict(source=frame, conf=0.5, stream=True)

    for r in results:
        # # Jalan
        # print("[DEBUG] Memproses hasil deteksi...") 

        # print("[DEBUG] r.obb:", r.obb)

        if r.obb is not None and hasattr(r.obb, "xywhr"):
            num_boxes = len(r.obb)
            print(f"[DEBUG] Total box terdeteksi: {num_boxes}")

            cls_list = r.obb.cls.cpu().numpy().astype(int)  # kelas
            conf_list = r.obb.conf.cpu().numpy()  # confidence
            names = model.names

            for i, cls in enumerate(cls_list):
                nama_terdeteksi = names[cls] if cls in names else f"Unknown-{cls}"
                # print(f"[DEBUG] Deteksi kelas {cls} → '{nama_terdeteksi}' (conf={conf_list[i]:.2f})")

                if nama_terdeteksi in daftar_anak:
                    if not sudah_terdeteksi(nama_terdeteksi):
                        tandai_hadir(nama_terdeteksi)
                        print(f"[INFO] {nama_terdeteksi} TERDETEKSI - disimpan ke database")
                    else:
                        print(f"[INFO] {nama_terdeteksi} sudah tercatat sebelumnya.")
                else:
                    print(f"[WARNING] '{nama_terdeteksi}' tidak ada di daftar_anak.")
        else:
            print("[DEBUG] Tidak ada box terdeteksi di frame ini.")


        # Tampil plotingan di frame
        annotated_frame = r.plot()

        # Tabel presensi (Yang item di dalem plot)
        cv2.rectangle(annotated_frame, (10, 10), (500, 10 + 30 + 25 * len(daftar_anak)), (0, 0, 0), -1)
        cv2.putText(annotated_frame, "Presensi Siswa", (20, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        y_offset = 60
        for nama in sorted(daftar_anak):
            status = "Hadir" if sudah_terdeteksi(nama) else "Belum Hadir"

            warna = (0, 255, 0) if status == "Hadir" else (0, 0, 255)
            cv2.putText(annotated_frame, f"{nama} - {status}", (20, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.55, warna, 2)
            y_offset += 25

        # Tampilkan dan simpan frame
        cv2.imshow("YOLOv8 Kehadiran", annotated_frame)
        output_path = "/home/abe-tanu/Documents/Code/Python/Artificial-Intelligence/presensi_obb/frame_hasil.jpg"
        cv2.imwrite(output_path, annotated_frame)
        print(f"[DEBUG] Frame disimpan di: {output_path}")

    # Tekan 'q' untuk keluar
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Print akhir
print("\n--- Rekap Kehadiran ---")
print("Hadir:")
for nama in sorted(daftar_anak):
    if sudah_terdeteksi(nama):
        print(nama)

print("Tidak Hadir:")
for nama in sorted(daftar_anak):
    if not sudah_terdeteksi(nama):
        print(nama)


# Simpan ke file
with open("/home/abe-tanu/Documents/Code/Python/Artificial-Intelligence/presensi_obb/hasil_hadir.txt", "w") as f:
    f.write("Hadir:\n")
    for nama in sorted(daftar_anak):
        if sudah_terdeteksi(nama):
            f.write(f"{nama}\n")

with open("/home/abe-tanu/Documents/Code/Python/Artificial-Intelligence/presensi_obb/hasil_tidak_hadir.txt", "w") as f:
    f.write("Tidak Hadir:\n")
    for nama in sorted(daftar_anak):
        if not sudah_terdeteksi(nama):
            f.write(f"{nama}\n")

# Tutup
cap.release()
cv2.destroyAllWindows()
clear_output(wait=True)
print(f"[DEBUG] Frame disimpan di: {output_path}")
conn.close()

[DEBUG] Frame disimpan di: /home/abe-tanu/Documents/Code/Python/Artificial-Intelligence/presensi_obb/frame_hasil.jpg


***SQLite Table***

In [17]:
# 1. Daftar anak-anak
daftar_anak = ["Adriel Bernhard T", "Jonea Kristiawan", "Kevin Tanwiputra", "Kevin Jiovanni Kuslin"]

# 2. Koneksi ke database
conn = sqlite3.connect("presensi.db")
c = conn.cursor()

# 3. Ambil nama dan waktu dari yang sudah hadir
c.execute("SELECT nama, waktu FROM presensi WHERE status = 'Hadir'")
hasil = c.fetchall()
nama_hadir = set(nama for nama, _ in hasil)  # untuk keperluan tidak_hadir

# 4. Cek siapa yang tidak hadir
tidak_hadir = [nama for nama in daftar_anak if nama not in nama_hadir]

# 5. Tampilkan
print("Yang HADIR:")
for nama, waktu in sorted(hasil):  # sorted by nama (bisa juga pakai sorted by waktu kalau mau)
    print(f"- {nama} (pukul -> {waktu})")

print("\nYang TIDAK HADIR:")
for nama in sorted(tidak_hadir):
    print(f"- {nama}")

# 6. Tutup koneksi
conn.close()

Yang HADIR:
- Adriel Bernhard T (pukul -> 2025-07-13 10:46:51)

Yang TIDAK HADIR:
- Jonea Kristiawan
- Kevin Jiovanni Kuslin
- Kevin Tanwiputra


***FLASK***